In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [2]:
data = pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


## Data preprocessing

In [3]:
## Drop irrelevant columns
data = data.drop(["RowNumber", "CustomerId", "Surname"], axis=1)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
## Data encoding, Categorical Variables
label_encoder = LabelEncoder()
data['Gender'] = label_encoder.fit_transform(data['Gender'])
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [ ]:
## Onehot encode geographic column

In [7]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse_output=False)
ohe_encoded_geo = ohe.fit_transform(data[['Geography']])
ohe.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [ ]:
ohe_encoded_geo_df = pd.DataFrame(ohe_encoded_geo, columns = ohe.get_feature_names_out(['Geography']))
##Combine Columns
data = pd.concat([data.drop('Geography', axis = 1), ohe_encoded_geo_df], axis = 1)
data.head()

In [ ]:
##Save the encoders and scalers

with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder, file)

with open('ohe_encoded_geo.pkl', 'wb') as file:
    pickle.dump(ohe, file)


In [ ]:
## Divide data

X = data.drop(['Exited'], axis = 1)
y = data['Exited']

In [ ]:
## Split data
X_train,X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

##Feature Scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
##Save scaler
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

## ANN Implementation

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input


## Build ANN model

In [ ]:
model = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(64, activation = 'relu'), # Hidden layer 1, connected with input layer
    Dense(32, activation = 'relu'), # HL2
    Dense(1, activation = 'sigmoid') # Output layer
    
])

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.optimizers import Adam
opt = Adam(learning_rate = 0.01) 
loss = tf.keras.losses.BinaryCrossentropy()

In [ ]:
## Compile the model
# Can create separate variables as shown above or can directly give string values for loss and optimizer
model.compile(optimizer="adam", loss = "binary_crossentropy", metrics = ['accuracy'])

In [ ]:
## Setup tensorboard
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime
#Log directory
log_dir = "logs/fit/"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback= TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
## Set up early stopping
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [ ]:
## Training the model
history = model.fit(
    X_train, y_train, validation_data=(X_test, y_test), epochs=100,
    callbacks=[tensorflow_callback, early_stopping_callback]
)

In [ ]:
model.save('model.h5') 
model.save('model.keras')

In [ ]:
# LOad ternsorboard
%load_ext tensorboard

In [ ]:
%tensorboard --logdir logs/fit